# Investigating Titanic Database

* Dataset [https://www.udacity.com/api/nodes/5420148578/supplemental_media/titanic-datacsv/download]
* Data description [https://www.kaggle.com/c/titanic/data]

## 1) Introdution
As we saw in the film directed by James Cameron that chronicles the Titanic's crash, women and children are a priority for a rescue. In this context, this study aims to analyze the database of ship's passengers and verify that the correlation between gender, age and survival. As an additional study, the correlation between income and passenger survival will be analyzed. It will be done from the information if the tickets bought were first, second or third class.
The questions that will be answered:
* Did age and gender influence the passenger's survival?
* Did passenger's class influenced passenger survival?

## 2) Variables
* <b>Dependent: </b> Passenger's survival
* <b>Independent: </b> Passenger's gender, age and class

## 3) Analysis

### 3.1) Meeting the data

In [8]:
import numpy as np
import pandas as pd
%matplotlib inline
-
titanic_file = 'titanic_data.csv'
titanic_data = pd.read_csv(titanic_file)

titanic_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
